# **Bloque 10 <br> Numpy**

### **Nombre del proyecto:** block10-02-pandas-01

#### **1. Datasets:**

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

- trade_details: dataset original con datos reales de operaciones financieras.
- trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

In [ ]:
import pandas as pd

trade_details = pd.read_csv("trade_details.csv",delimiter=";")
trade_details_snapshot = pd.read_csv("trade_details_snapshot.csv",delimiter=";")


display(trade_details)
display(trade_details_snapshot)


#### **2. Columnas y significado:**

- mfamily: indica la familia de operaciones a la que pertenece.
- mgroup: indica el grupo de operaciones dentro de la familia.
- mtype: indica el tipo de operación dentro del grupo.
- origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
- origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
- maturity: fecha de finalización del contrato de cada operación.

#### **3. Descripción del problema:**

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg*.

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

#### **4. Reto:**

- Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
- Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
- Hacerlo de la manera más eficiente posible a nivel computacional.

In [ ]:
# 1. Filtrar las operaciones tipo FXSwap

fxswaps = trade_details[
    (trade_details['mfamily'] == 'CURR') &
    (trade_details['mgroup'] == 'FXD') &
    (trade_details['mtype'] == 'SWLEG')
]

display(fxswaps)


# 2. Filtrar las operaciones tipo FXSwap del trade_details_snapshot sacando solo las columnas que nos interesan

fxswaps_snapshot = trade_details_snapshot[
    (trade_details_snapshot['mfamily'] == 'CURR') &
    (trade_details_snapshot['mgroup'] == 'FXD') &
    (trade_details_snapshot['mtype'] == 'SWLEG')
][['origin_trade_number', 'origin_contract_number', 'maturity']]

display(fxswaps_snapshot)


# 3. Con merge() how='left' se conservarán todas las filas del DataFrame izquierdo (fxswaps) y solo se agregarán
# cuando haya coincidencia en las columnas on= ('origin_trade_number' y 'origin_contract_number')
# Si no hay coincidencia se agregarán valores nulos (NaN).

merged_fxswaps = fxswaps.merge(fxswaps_snapshot, on=[
                               'origin_trade_number', 'origin_contract_number'], how='left')

display(merged_fxswaps)


# 4. Se aplica fillna() en la columna 'maturity_x' del DataFrame merged_fxswaps llamando a maturity_y'.
# inplace=True indica que el DataFrame (merged_fxswaps) será modificado.
# Es decir, los NAN de 'maturity_x' se reemplazarán con los'maturity_y' sin crear un nuevo DataFrame.
# Si se establece False o se omite, se devolverá un nuevo DataFramev con los valores modificados y el DataFrame original no se modificará.

merged_fxswaps['maturity_x'].fillna(merged_fxswaps['maturity_y'], inplace=True)

display(merged_fxswaps)


# 5. Se elimina la columna maturity_y'.

merged_fxswaps.drop('maturity_y', axis=1, inplace=True)
display(merged_fxswaps)


# 6. Se renombra la columna maturity_x y se resetean los índices. .
# inplace=True para modificar merged_fxswaps y que no cree uno nuevo

merged_fxswaps.rename(columns={'maturity_x': 'maturity'}, inplace=True)

display(merged_fxswaps)


# 7.Convertir la columna "maturity" a formato datetime si no está en ese formato para realizar operaciones de filtrado
merged_fxswaps['maturity'] = pd.to_datetime(merged_fxswaps['maturity'])


# 8. Ordenar el DataFrame por "origin_contract_number" y "maturity" de forma ascendente según origin y descendente para maturity
merged_fxswaps.sort_values(by=['origin_contract_number', 'maturity'], ascending=[
                           True, False], inplace=True)

display(merged_fxswaps)


# 9. Eliminar las filas duplicadas en función de "origin_contract_number" manteniendo solo la primera (la más reciente)
df_final = merged_fxswaps.drop_duplicates(
    subset='origin_contract_number', keep='first').reset_index(drop=True)


# 10. df_final
display(df_final)
